In [1]:
import pandas as pd

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go   

In [29]:
from numba import njit
from numba import jit
from window_ops.expanding import expanding_mean
from window_ops.rolling import rolling_mean

@njit
def rolling_mean_14(x):
    return rolling_mean(x, window_size=14)
@njit
def rolling_mean_30(x):
    return rolling_mean(x, window_size=30)

In [3]:
def format_df_to_mlforecast(df, date_col, target_col, unique_id='mean'):
    df_ = df.rename({
        date_col: "ds",
        # target_col: 'y',
    }, axis=1)

    df_['ds'] = pd.to_datetime(df_['ds'])

    df_['y'] = df_[target_col].copy()
    # df_.drop(columns=target_col)

    df_['unique_id'] = unique_id
    return df_

In [4]:
selected_sensors_df = pd.read_csv("../data/selected_sensors2_cleaned.csv", index_col=0)

In [5]:
scenarios_sensors = {
    # 0: 1, 4372603
    # "0_12M_train_7M_test": {"train_start": "2017-03-25", "train_end": "2018-03-25", "test_start": "2018-03-26", "test_end": "2018-10-10"},
    '2': {
        "0_10M_train_9M_test":  {"train_start": "2017-04-01", "train_end": "2018-01-25", "test_start": "2018-01-26", "test_end": "2018-10-10"},
        "0_8M_train_11M_test":  {"train_start": "2017-04-01", "train_end": "2017-10-25", "test_start": "2017-10-26", "test_end": "2018-10-10"},
        
        # Non-Heating Periods
        "0_NonHeating_3M_train_3M_test":  {"train_start": "2017-04-15", "train_end": "2017-07-15", "test_start": "2017-07-16", "test_end": "2017-10-01"},
        "0_NonHeating_4M_train_2M_test":  {"train_start": "2017-04-15", "train_end": "2017-08-15", "test_start": "2017-08-16", "test_end": "2017-10-01"},
        "0_NonHeating_2M_train_4M_test":  {"train_start": "2017-04-15", "train_end": "2017-06-15", "test_start": "2017-06-16", "test_end": "2017-10-01"},
        "0_NonHeating_1M_train_5M_test":  {"train_start": "2017-04-15", "train_end": "2017-05-15", "test_start": "2017-05-16", "test_end": "2017-10-01"},
        "0_NonHeating_15D_train_5M_test": {"train_start": "2017-04-15", "train_end": "2017-04-30", "test_start": "2017-05-01", "test_end": "2017-10-01"},
        "0_NonHeating_feb_2M_train_4M_test": {"train_start": "2017-02-15", "train_end": "2017-04-15", "test_start": "2017-04-16", "test_end": "2017-08-16"},
        "0_NonHeating_feb_1M_train_4M_test": {"train_start": "2017-02-15", "train_end": "2017-04-15", "test_start": "2017-04-16", "test_end": "2017-08-16"},
        "0_NonHeating_mar_2M_train_4M_test": {"train_start": "2017-03-15", "train_end": "2017-05-15", "test_start": "2017-05-16", "test_end": "2017-09-16"},
        "0_NonHeating_mar_1M_train_4M_test": {"train_start": "2017-03-15", "train_end": "2017-04-15", "test_start": "2017-05-16", "test_end": "2017-09-16"},

        # Heating Periods
        "0_Heating_5M_train_1Y_test":     {"train_start": "2017-06-01", "train_end": "2017-11-01", "test_start": "2017-11-02", "test_end": "2018-10-10"},
        "0_Heating_3M_jul_train_1Y_test": {"train_start": "2017-07-01", "train_end": "2017-10-10", "test_start": "2017-10-11", "test_end": "2018-10-10"},
        "0_Heating_3M_sep_train_1Y_test": {"train_start": "2017-09-01", "train_end": "2017-12-10", "test_start": "2017-12-11", "test_end": "2018-12-10"},
        "0_Heating_3M_nov_train_1Y_test": {"train_start": "2017-11-01", "train_end": "2018-02-10", "test_start": "2018-02-11", "test_end": "2018-12-10"},
        },
}
scenarios_sensors['5'] = scenarios_sensors['2'].copy()
scenarios_sensors['6'] = scenarios_sensors['2'].copy()

In [6]:
from MLForecastPipeline import *

In [7]:
def get_seasonal_data(df, start_date, end_date, date_col="ds"):
    """Filters data for a specific seasonal period."""
    return df[(df[date_col] >= start_date) & (df[date_col] <= end_date)]

def split_data(df, scenario, date_col="ds"):
    """Extracts train and test data based on a given time window scenario."""
    train_data = get_seasonal_data(df, scenario["train_start"], scenario["train_end"], date_col)
    test_data = get_seasonal_data(df, scenario["test_start"], scenario["test_end"], date_col)
    return train_data, test_data

models = {
    # "XGBRegressor": XGBRegressor(),
    # "SGDRegressor_42": SGDRegressor(random_state=42),
    # "SGDRegressor_1": SGDRegressor(random_state=1),
    # "Ridge": Ridge(),
    "Lasso": Lasso()
}

# Define lag transformations

lag_transforms_options = [
    {1: [expanding_mean], 7: [rolling_mean_14], 30: [expanding_mean]},
    # {1: [rolling_mean_14], 7: [rolling_mean_30], 30: [expanding_mean]},
    # {1: [rolling_mean_14], 30: [expanding_mean]},
    # {1: [rolling_mean_14]},
    # {},
]

In [ ]:
# from tqdm.notebook import trange, tqdm
# from time import sleep

# for i in trange(3, desc='1st loop'):
#     for j in tqdm(range(100), desc='2nd loop'):
#         sleep(0.01)

## parallel

In [8]:
from joblib import Parallel, delayed
import time

In [20]:
sensor_name = '2'
scenarios_sensors = {
    # 0: 1, 4372603
    # "0_12M_train_7M_test": {"train_start": "2017-03-25", "train_end": "2018-03-25", "test_start": "2018-03-26", "test_end": "2018-10-10"},
    '2': {
        # Non-Heating Periods
        "0_NonHeating_3M_train_3M_test":  {"train_start": "2017-04-15", "train_end": "2017-07-15", "test_start": "2017-07-16", "test_end": "2017-10-01"},
        "0_NonHeating_4M_train_2M_test":  {"train_start": "2017-04-15", "train_end": "2017-08-15", "test_start": "2017-08-16", "test_end": "2017-10-01"},
        "0_NonHeating_2M_train_4M_test":  {"train_start": "2017-04-15", "train_end": "2017-06-15", "test_start": "2017-06-16", "test_end": "2017-10-01"},
        "0_NonHeating_1M_train_5M_test":  {"train_start": "2017-04-15", "train_end": "2017-05-15", "test_start": "2017-05-16", "test_end": "2017-10-01"},
        },
}
models = {
    "SGDRegressor_42": SGDRegressor(random_state=42),
    "SGDRegressor_1": SGDRegressor(random_state=1),
    "Ridge": Ridge(),
    "Lasso": Lasso()
}

# Define lag transformations

lag_transforms_options = [
    {1: [expanding_mean], 7: [rolling_mean_14], 30: [expanding_mean]},
]

# Loop through scenarios and evaluate models
start = time.perf_counter()

results = []
for sensor_name, scenarios in scenarios_sensors.items():
    formatted_df = format_df_to_mlforecast(selected_sensors_df[['full_date', sensor_name]], 'full_date', sensor_name, unique_id=sensor_name)
    formatted_df = formatted_df[['ds', 'y', 'unique_id']]

    for scenario_name, scenario in scenarios.items():

        train_df, test_df = split_data(formatted_df, scenario)

        optimal_lags_list = get_optimal_lags(train_df, 'y', 
                                            ratios=[1]
                                            # ratios=[0.33, 0.66, 1]
                                            #  ratios=[0.25, 0.5, 0.75, 1]
        )
        target_transforms = get_dynamic_transforms(train_df)
        results = evaluate_models(train_df, test_df, models, target_transforms, lag_transforms_options, optimal_lags_list)

        save_results(results, f"results/{scenario_name}.csv") 

end = time.perf_counter()

Total model fits to run: 192
0/192 Training SGDRegressor_42 with transforms (), lags [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], and lag_transforms {1: [CPUDispatcher(<function expanding_mean at 0x000001A49F02FCE0>)], 7: [CPUDispatcher(<function rolling_mean_14 at 0x000001A49E682A20>)], 30: [CPUDispatcher(<function expanding_mean at 0x000001A49F02FCE0>)]}...
Skipping combination 0 due to error: Input X contains infinity or a value too large for dtype('float64').
1/192 Training SGDRegressor_1 with transforms (), lags [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], and lag_transforms {1: [CPUDispatcher(<function expanding_mean at 0x000001A49F02FCE0>)], 7: [CPUDispatcher(<function rolling_mean_14 at 0x000001A49E682A20>)], 30: [CPUDispatcher(<function expanding_mean at 0x000001A49F02FCE0>)]}...


c:\Users\77019\pyver\py312\Lib\site-packages\sklearn\utils\extmath.py:208: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
c:\Users\77019\pyver\py312\Lib\site-packages\sklearn\utils\extmath.py:208: RuntimeWarning: overflow encountered in matmul
  ret = a @ b


Skipping combination 1 due to error: Input X contains infinity or a value too large for dtype('float64').
2/192 Training Ridge with transforms (), lags [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], and lag_transforms {1: [CPUDispatcher(<function expanding_mean at 0x000001A49F02FCE0>)], 7: [CPUDispatcher(<function rolling_mean_14 at 0x000001A49E682A20>)], 30: [CPUDispatcher(<function expanding_mean at 0x000001A49F02FCE0>)]}...
Ridge MAPE: 21.79% with transforms (), lags [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], and lag_transforms {1: [CPUDispatcher(<function expanding_mean at 0x000001A49F02FCE0>)], 7: [CPUDispatcher(<function rolling_mean_14 at 0x000001A49E682A20>)], 30: [CPUDispatcher(<function expanding_mean at 0x000001A49F02FCE0>)]}
3/192 Training Lasso with transforms (), lags [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], and lag_transforms {1: [CPUDispatcher(<function expanding_mean at 0x000001A49F

c:\Users\77019\pyver\py312\Lib\site-packages\sklearn\utils\extmath.py:208: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
c:\Users\77019\pyver\py312\Lib\site-packages\sklearn\utils\extmath.py:208: RuntimeWarning: overflow encountered in matmul
  ret = a @ b


Skipping combination 9 due to error: Input X contains infinity or a value too large for dtype('float64').
10/192 Training Ridge with transforms (<mlforecast.target_transforms.AutoSeasonalDifferences object at 0x000001A4B1AB2E10>,), lags [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], and lag_transforms {1: [CPUDispatcher(<function expanding_mean at 0x000001A49F02FCE0>)], 7: [CPUDispatcher(<function rolling_mean_14 at 0x000001A49E682A20>)], 30: [CPUDispatcher(<function expanding_mean at 0x000001A49F02FCE0>)]}...
Ridge MAPE: 21.79% with transforms (<mlforecast.target_transforms.AutoSeasonalDifferences object at 0x000001A4B1AB2E10>,), lags [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], and lag_transforms {1: [CPUDispatcher(<function expanding_mean at 0x000001A49F02FCE0>)], 7: [CPUDispatcher(<function rolling_mean_14 at 0x000001A49E682A20>)], 30: [CPUDispatcher(<function expanding_mean at 0x000001A49F02FCE0>)]}
11/192 Training Lasso with t

...

Skipping combination 124 due to error: Input X contains infinity or a value too large for dtype('float64').
125/128 Training SGDRegressor_1 with transforms (<mlforecast.target_transforms.AutoSeasonalityAndDifferences object at 0x000001A4B1A888C0>, <mlforecast.target_transforms.LocalBoxCox object at 0x000001A4B2DFF3E0>), lags [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], and lag_transforms {1: [CPUDispatcher(<function expanding_mean at 0x000001A49F02FCE0>)], 7: [CPUDispatcher(<function rolling_mean_14 at 0x000001A49E682A20>)], 30: [CPUDispatcher(<function expanding_mean at 0x000001A49F02FCE0>)]}...


c:\Users\77019\pyver\py312\Lib\site-packages\sklearn\utils\extmath.py:208: RuntimeWarning: overflow encountered in matmul
  ret = a @ b


Skipping combination 125 due to error: Input X contains infinity or a value too large for dtype('float64').
126/128 Training Ridge with transforms (<mlforecast.target_transforms.AutoSeasonalityAndDifferences object at 0x000001A4B1A888C0>, <mlforecast.target_transforms.LocalBoxCox object at 0x000001A4B2DFF3E0>), lags [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], and lag_transforms {1: [CPUDispatcher(<function expanding_mean at 0x000001A49F02FCE0>)], 7: [CPUDispatcher(<function rolling_mean_14 at 0x000001A49E682A20>)], 30: [CPUDispatcher(<function expanding_mean at 0x000001A49F02FCE0>)]}...
Ridge MAPE: 35.66% with transforms (<mlforecast.target_transforms.AutoSeasonalityAndDifferences object at 0x000001A4B1A888C0>, <mlforecast.target_transforms.LocalBoxCox object at 0x000001A4B2DFF3E0>), lags [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], and lag_transforms {1: [CPUDispatcher(<function expanding_mean at 0x000001A49F02FCE0>)], 7: [CPUDi

c:\Users\77019\pyver\py312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 0.000e+00, tolerance: 0.000e+00
  model = cd_fast.enet_coordinate_descent(


Lasso MAPE: 35.66% with transforms (<mlforecast.target_transforms.AutoSeasonalityAndDifferences object at 0x000001A4B1A888C0>, <mlforecast.target_transforms.LocalBoxCox object at 0x000001A4B2DFF3E0>), lags [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], and lag_transforms {1: [CPUDispatcher(<function expanding_mean at 0x000001A49F02FCE0>)], 7: [CPUDispatcher(<function rolling_mean_14 at 0x000001A49E682A20>)], 30: [CPUDispatcher(<function expanding_mean at 0x000001A49F02FCE0>)]}
Results saved to results/0_NonHeating_1M_train_5M_test.csv


In [21]:
print(f"Sequential execution time: {end - start:.2f} seconds")

Sequential execution time: 298.48 seconds


In [22]:
def parallel_evaluate_models(train_df, test_df, models, target_transforms, lag_transforms_options, optimal_lags_list):
    start_time = time.perf_counter()

    results = Parallel(n_jobs=-1)(
        delayed(evaluate_models)(
            train_df, test_df, {model_name: model}, target_transforms, lag_transforms_options, optimal_lags_list
        )
        for model_name, model in models.items()
    )

    end_time = time.perf_counter()
    print(f"Parallel execution time: {end_time - start_time:.2f} seconds")

    return pd.concat(results, ignore_index=True)

In [23]:
from joblib import Parallel, delayed
import time

def process_scenario(sensor_name, scenario_name, scenario, selected_sensors_df, models, lag_transforms_options):
    """ Process each scenario independently and save results. """
    start_time = time.perf_counter()

    formatted_df = format_df_to_mlforecast(selected_sensors_df[['full_date', sensor_name]], 'full_date', sensor_name, unique_id=sensor_name)
    formatted_df = formatted_df[['ds', 'y', 'unique_id']]
    
    train_df, test_df = split_data(formatted_df, scenario)
    optimal_lags_list = get_optimal_lags(train_df, 'y', ratios=[1])
    target_transforms = get_dynamic_transforms(train_df)

    results = evaluate_models(train_df, test_df, models, target_transforms, lag_transforms_options, optimal_lags_list)

    # Save results
    save_results(results, f"results/{scenario_name}.csv")

    end_time = time.perf_counter()
    print(f"Scenario {scenario_name} completed in {end_time - start_time:.2f} seconds")

    return results

def run_all_scenarios_parallel(scenarios_sensors, selected_sensors_df, models, lag_transforms_options):
    start_time = time.perf_counter()

    # Parallel processing for scenarios
    results = Parallel(n_jobs=-1)(
        delayed(process_scenario)(sensor_name, scenario_name, scenario, selected_sensors_df, models, lag_transforms_options)
        for sensor_name, scenarios in scenarios_sensors.items()
        for scenario_name, scenario in scenarios.items()
    )

    end_time = time.perf_counter()
    print(f"Total parallel execution time: {end_time - start_time:.2f} seconds")

    return results


In [24]:
run_all_scenarios_parallel(scenarios_sensors, selected_sensors_df, models, lag_transforms_options)
/

Total parallel execution time: 148.52 seconds


()

In [25]:
start = time.perf_counter()

results = []
for sensor_name, scenarios in scenarios_sensors.items():
    formatted_df = format_df_to_mlforecast(selected_sensors_df[['full_date', sensor_name]], 'full_date', sensor_name, unique_id=sensor_name)
    formatted_df = formatted_df[['ds', 'y', 'unique_id']]

    for scenario_name, scenario in scenarios.items():

        train_df, test_df = split_data(formatted_df, scenario)

        optimal_lags_list = get_optimal_lags(train_df, 'y', 
                                            ratios=[1]
                                            # ratios=[0.33, 0.66, 1]
                                            #  ratios=[0.25, 0.5, 0.75, 1]
        )
        target_transforms = get_dynamic_transforms(train_df)
        results = parallel_evaluate_models(train_df, test_df, models, target_transforms, lag_transforms_options, optimal_lags_list)

        save_results(results, f"results/{scenario_name}.csv") 

end = time.perf_counter()

Parallel execution time: 74.44 seconds
Results saved to results/0_NonHeating_3M_train_3M_test.csv
Parallel execution time: 36.80 seconds
Results saved to results/0_NonHeating_4M_train_2M_test.csv
Parallel execution time: 27.19 seconds
Results saved to results/0_NonHeating_2M_train_4M_test.csv
Parallel execution time: 26.04 seconds
Results saved to results/0_NonHeating_1M_train_5M_test.csv


parallelization by scenarios wins!!!

In [27]:
print(f"Total parallel (models) execution time: {end - start:.2f} seconds")

Total parallel (models) execution time: 167.71 seconds


## numba 

In [50]:
import numpy as np
import pandas as pd
import time
from numba import jit, prange
from window_ops.expanding import expanding_mean  # Import optimized method

# 🏆 Optimized Functions with Numba
@jit(nopython=True, parallel=True)
def fast_shift(arr, lags):
    """ Fast Numba version of lag feature computation. """
    n = arr.shape[0]
    result = np.full((n, len(lags)), np.nan)
    for i in prange(len(lags)):  
        lag = lags[i]
        if lag < n:
            result[lag:, i] = arr[:-lag]
    return result

@jit(nopython=True)
def fast_mape(y_true, y_pred):
    """ Fast MAPE calculation with Numba. """
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    n = len(y_true)
    error_sum = 0.0
    for i in range(n):
        error_sum += abs((y_true[i] - y_pred[i]) / (y_true[i] + 1e-9))
    return (error_sum / n) * 100

@jit(nopython=True)
def fast_expanding_mean(arr):
    """ Computes expanding mean using Numba. """
    n = arr.shape[0]
    result = np.empty(n)
    sum_val = 0.0
    for i in range(n):
        sum_val += arr[i]
        result[i] = sum_val / (i + 1)
    return result

# 📊 Benchmarking Function
def benchmark_function(func, *args):
    start = time.perf_counter()
    result = func(*args)
    end = time.perf_counter()
    return result, end - start  # Return result and execution time

# Generate large dataset
N = 1_000_000  # 1 million data points
arr = np.random.rand(N)
y_true = list(np.random.rand(N))
y_pred = list(np.random.rand(N))
lags = np.arange(1, 11)

# 🏆 Run Benchmarks and Compare
print("\n--- Benchmarking Lag Feature Generation ---")
_, pandas_time = benchmark_function(lambda x: pd.concat([pd.Series(x).shift(lag) for lag in lags], axis=1), arr)
_, numba_time = benchmark_function(fast_shift, arr, lags)
print(f"Pandas Shift: {pandas_time:.4f} sec")
print(f"Numba Shift: {numba_time:.4f} sec")
print(f"Speedup Factor: {pandas_time / numba_time:.2f}x\n")

print("\n--- Benchmarking MAPE Calculation ---")
_, numpy_time = benchmark_function(lambda y, p: np.mean(np.abs((np.array(y) - np.array(p)) / (np.array(y) + 1e-9))) * 100, y_true, y_pred)
_, numba_time = benchmark_function(fast_mape, y_true, y_pred)
print(f"NumPy MAPE: {numpy_time:.4f} sec")
print(f"Numba MAPE: {numba_time:.4f} sec")
print(f"Speedup Factor: {numpy_time / numba_time:.2f}x\n")

print("\n--- Benchmarking Expanding Mean ---")
_, window_ops_time = benchmark_function(expanding_mean, arr)
_, numba_time = benchmark_function(fast_expanding_mean, arr)
print(f"Window Ops Expanding Mean: {window_ops_time:.4f} sec")
print(f"Numba Expanding Mean: {numba_time:.4f} sec")
print(f"Speedup Factor: {window_ops_time / numba_time:.2f}x\n")



--- Benchmarking Lag Feature Generation ---
Pandas Shift: 0.1312 sec
Numba Shift: 1.3985 sec
Speedup Factor: 0.09x


--- Benchmarking MAPE Calculation ---
NumPy MAPE: 0.3219 sec
Numba MAPE: 6.0302 sec
Speedup Factor: 0.05x


--- Benchmarking Expanding Mean ---
Window Ops Expanding Mean: 0.0039 sec
Numba Expanding Mean: 0.1286 sec
Speedup Factor: 0.03x



In [54]:
train_df, test_df = split_data(formatted_df, scenario)


In [56]:
import pandas as pd
import numpy as np
from mlforecast import MLForecast
from sklearn.ensemble import RandomForestRegressor
from mlforecast.target_transforms import LocalStandardScaler
from window_ops.rolling import rolling_mean

model = RandomForestRegressor(n_estimators=10, random_state=42)
target_transforms = [LocalStandardScaler()]
date_features = ['dayofweek']
optimal_lags = [1, 2, 3]

# ✅ Initialize MLForecast
fcst = MLForecast(
    models=[model],
    freq='D',
    lags=optimal_lags,
    target_transforms=target_transforms,
    date_features=date_features,
    num_threads=1,
)

# ✅ Fit the model
fcst.fit(train_df)

# ✅ Make predictions
max_test_length = 5
predictions = fcst.predict(h=max_test_length)

# Convert predictions to Series for comparison
model_name = model.__class__.__name__
pred_series = predictions[model_name].reset_index(drop=True)

# ✅ Method 1: Using .copy()
test_df_copy = train_df.copy()
test_df_copy.loc[-max_test_length:, 'forecast'] = pred_series.values

# ✅ Method 2: Using .loc (modifies in place)
train_df.loc[:, 'forecast'] = pred_series.values

# ✅ Print results
print("\nOriginal DataFrame (Modified In-Place):")
print(train_df)

print("\nNew DataFrame (Copy):")
print(test_df_copy)

# ✅ Check if results are identical
same_result = train_df.equals(test_df_copy)
print(f"\nDo both methods produce the same result? {same_result}")


ValueError: could not broadcast input array from shape (5,) into shape (31,)

## generators

In [ ]:
start = time.perf_counter()

results = []
for sensor_name, scenarios in scenarios_sensors.items():
    formatted_df = format_df_to_mlforecast(selected_sensors_df[['full_date', sensor_name]], 'full_date', sensor_name, unique_id=sensor_name)
    formatted_df = formatted_df[['ds', 'y', 'unique_id']]

    for scenario_name, scenario in scenarios.items():

        train_df, test_df = split_data(formatted_df, scenario)

        optimal_lags_list = get_optimal_lags(train_df, 'y', 
                                            ratios=[1]
                                            # ratios=[0.33, 0.66, 1]
                                            #  ratios=[0.25, 0.5, 0.75, 1]
        )
        target_transforms = get_dynamic_transforms(train_df)
        results = parallel_evaluate_models(train_df, test_df, models, target_transforms, lag_transforms_options, optimal_lags_list)

        save_results(results, f"results/{scenario_name}.csv") 

end = time.perf_counter()

In [ ]:
# Loop through scenarios and evaluate models using generators
for sensor_name, scenarios in scenarios_sensors.items():
    formatted_df = format_df_to_mlforecast(selected_sensors_df[['full_date', sensor_name]], 'full_date', sensor_name, unique_id=sensor_name)
    formatted_df = formatted_df[['ds', 'y', 'unique_id']]

    for scenario_name, scenario in scenarios.items():
        train_df, test_df = split_data(formatted_df, scenario)

        # Use generator for optimal lags
        optimal_lags_gen = optimal_lags(train_df, 'y', ratios=[1])

        # Save results incrementally
        save_results_generator(
            evaluate_models_generator(train_df, test_df, models, target_transforms, lag_transforms_options, optimal_lags_gen),
            f"results/{scenario_name}.csv"
        )
